# Provingground - HoTT

These notes concern the object _HoTT_, which has the core implementation of homotopy type theory. Implementation details are (rather, will be) in the [scaladocs](http://siddhartha-gadgil.github.io/ProvingGround/).

The major components of homotopy type theory implemented in the object HoTT are

* Terms, types and Universes.
* Function and dependent function types.
* λs.
* Pairs and Dependent pairs.
* Disjoint union types.
* Types 0 and 1 and an object in the latter.
* Identity types

Inductive types, induction and recursion are in different objects as they are rather subtle. The other major way (also not in the _HoTT_ object) of constructing non-composite types is to wrap scala types, possibly including symbolic algebra.


The _core_ project contains code that is agnostic to how it is run. In particular this also compiles to scala-js.

In [1]:
load.jar("/home/gadgil/code/ProvingGround/core/.jvm/target/scala-2.11/ProvingGround-Core-assembly-0.8.jar")

### Universes, Symbolic types

We have a family of universes, but mostly use the first one denoted by a double underscore. Given a type, we can construct symbolic objects of that type. We construct such a type _A_.

In [2]:
import provingground._
import HoTT._
val A ="A" :: __

import provingground._
import HoTT._
A: provingground.HoTT.Typ[provingground.HoTT.Term] with provingground.HoTT.Subs[provingground.HoTT.Typ[provingground.HoTT.Term]] = A : 𝒰 

We consider a symbolic object of the type _A_

In [3]:
val a ="a" :: A

a: Term with Subs[Term] = a : (A : 𝒰 )

## Function types, lambdas, Identity

Given types A and B, we have the function type A → B. An element of this is a function from A to B. 

We can construct functions using λ's. Here, for the type _A_, we construct the identity on _A_ using a lambda. We can then view this as a dependent function of _A_, giving the identity function.

In this definition, two λ's are used, with the method _lmbda_ telling the __compiler__ that the result is a (non-dependent) function. 

In [4]:
val id = lambda(A)(lmbda(a)(a))

id: FuncLike[Typ[Term] with Subs[Typ[Term]], Func[Term with Subs[Term], Term with Subs[Term]]] = (A : 𝒰 ) ↦ ((a : (A : 𝒰 )) ↦ (a : (A : 𝒰 )))

The type of the identity function is a mixture of Pi-types and function types. Which of these to use is determined by checking dependence of the type of the value on the varaible in a λ-definition.

In [5]:
id.typ
lmbda(a)(a).typ
lmbda(a)(a).typ.dependsOn(A)

res4_0: Typ[Term] = ∏((A : 𝒰 ) ↦ ((A : 𝒰 ) → (A : 𝒰 )))
res4_1: Typ[Term] = (A : 𝒰 ) → (A : 𝒰 )
res4_2: Boolean = true

The lambdas have the same effect at runtime. It is checked if the type of the value depends on the variable.
The result is either _LambdaFixed_ or _Lambda_ accordingly.

In [ ]:
val indep = lmbda(a)(a)
val dep = lambda(a)(a)
indep == dep

### Hygiene for λs

A new variable object, which has the same toString, is created in making lambdas. This is to avoid name clashes.

In [ ]:
val l = dep.asInstanceOf[LambdaFixed[Term, Term]]
l.variable
l.variable == a

We can construct Modus Ponens. Note that _A ->: B_ is the function type

In [ ]:
val B = "B" :: __
val f = "f" :: (A ->: B)
val mp = lambda(A)(lambda(B)(lmbda(a)(lmbda(f)(f(a)))))

We can apply modus ponens with the roles of _A_ and _B_ reversed. This still works because variable clashes are avoided.

In [ ]:
val mpBA = mp(B)(A)
mpBA.typ == B ->: (B ->: A) ->: A

Lambdas do not depend on the name of the variable.

In [ ]:
val aa = "aa" :: A
lmbda(aa)(aa) == lmbda(a)(a)
(lmbda(aa)(aa))(a) == a

We consider a type family, and construct a Pi-Type. We are using lambdas for the family. Note that the !: method just claims and checks a type, and is useful (e.g. here) for documentation.

In [ ]:
val Bs = "B(_ : A)" :: (A ->: __)

In [ ]:
val fmly = (a !: A) ~>: (Bs(a) ->: A)

There is also a convenience method for defining Sigma types using lambdas

In [ ]:
Sgma(a !: A, Bs(a))

In [ ]:
Sgma(a !: A, Bs(a) ->: Bs(a) ->: A)